# Loads Data

In [204]:
# Based off of Neural Graph Collaborative Filtering
 # Xiang Wang, Xiangnan He, Meng Wang, Fuli Feng, and Tat-Seng Chua. 2019.
 # Neural Graph Collaborative Filtering. In Proceedings of the 42nd International
 # ACM SIGIR Conference on Research and Development in Information Retrieval
 # (SIGIR ’19), July 21–25, 2019, Paris, France. ACM, New York, NY, USA,
 # 10 pages. https://doi.org/10.1145/3331184.3331267

import torch
import random
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.nn import init, LeakyReLU, Linear, Module, ModuleList, Parameter

batch_size = 1024 # Set as such because it says this is the optimal number in the above paper

# Gets values from csv
input = pd.read_csv("ratings_small.csv", usecols=["userId", "movieId", "rating"])


In [205]:
# Splits input into training and testing sets
userSet = []
spot = 1
activeItems = []
for a, b, c in input.values:
    if spot == a:
        activeItems.append(b)
    else:
        userSet.append(activeItems)
        activeItems = []
        spot += 1
        activeItems.append(b)
userSet.append(activeItems)

training = []
testing = []
for i in range(len(userSet)):
    length = int(len(userSet[i])*0.7)
    for a in userSet[i][:length]:
        training.append([i+1, a])
    for a in userSet[i][length:]:
        testing.append([i+1, a])


In [206]:
# Gets lists of unique values
userIDList = input['userId'].unique()
movieIDList = input['movieId'].unique()

In [207]:
def get_user_item_matrix(matrix):
    output = pd.DataFrame(input, columns=movieIDList, index=userIDList)
    for a, b in matrix:
        output.at[a, b] = 1
    output.fillna(value = 0, inplace = True)
    output = torch.from_numpy(output.values)
    return output

# Builds the user item interaction matrix (training)
userItemIteractionMatrix = get_user_item_matrix(training)
# Builds the user item interaction matrix (testing)
testingUIMatrix = get_user_item_matrix(testing)

In [208]:
# Builds the positive list
def build_positive_list(matrix):
    positiveList = []
    spot = 1
    temp = []
    for a, b in matrix:
        b = int(b)
        if a == spot:
            temp.append(b)
        else:
            spot += 1
            positiveList.append(temp)
            temp = [b]
    positiveList.append(temp)
    return positiveList

positiveList = build_positive_list(training)

# Creates Adjacency Matrix

In [209]:
numberOfUsers = len(userIDList)
numberOfItems = len(movieIDList)

AdjanceyMatrix = np.zeros((numberOfUsers + numberOfItems, numberOfItems + numberOfUsers))
AdjanceyMatrix[:numberOfUsers, numberOfUsers:] = userItemIteractionMatrix.tolist()
AdjanceyMatrix[numberOfUsers:, :numberOfUsers] = userItemIteractionMatrix.T.tolist()

rowsum = np.array(AdjanceyMatrix.sum(1))

# Tecnique taken from the paper's implementaion instead of the paper itself
 # https://github.com/xiangwang1223/neural_graph_collaborative_filtering
d = np.power(rowsum, -1).flatten()
d[np.isinf(d)] = 0.
d_mat = np.zeros((len(d), len(d)))
for i in range(len(d)):
    d_mat[i][i] = d[i]
NormalizedAdjacencyMatrix = d_mat.dot(AdjanceyMatrix)
stuff = torch.from_numpy(AdjanceyMatrix).to_sparse()
AdjanceyMatrix = NormalizedAdjacencyMatrix + np.eye(NormalizedAdjacencyMatrix.shape[0])
sparceAdjacenyMatrix = torch.from_numpy(AdjanceyMatrix.astype(np.float32)).to_sparse()

C:\Users\jgill\AppData\Local\Temp\ipykernel_15280\2393360537.py:12: RuntimeWarning: divide by zero encountered in power
  d = np.power(rowsum, -1).flatten()


In [210]:

def positive(u):
    movie = random.randint(0, len(positiveList[u-1])-1)
    for i in range(len(movieIDList)):
        if movieIDList[i] == positiveList[u-1][movie]:
            return i

def negative(u):
    while True:
        movie = random.randint(0, numberOfItems-1)
        movie = movieIDList[movie]
        if movie not in positiveList[u-1]:
            for i in range(len(movieIDList)):
                if movieIDList[i] == movie:
                    return i

def get_a_sample():
    selectedUsers = np.random.choice(userIDList, size=batch_size)
    pos_sample, neg_sample = [], []
    for u in selectedUsers:
        pos_sample.append(positive(u))
        neg_sample.append(negative(u))
    return selectedUsers, np.asarray(pos_sample), np.asarray(neg_sample)


# The Model

In [211]:
numberOfLayers = 3

transfomationSize = 64

class NGCF(Module):
  def __init__(self):
    super().__init__()

    # This portion is adapted from https://github.com/SytzeAndr/NGCF_RP32/blob/hand-in/NGCF.ipynb since
     # I still don't understand how Parameter works in torch
    self.initialUserEmbeddings = Parameter(self.createRandomEmbeddings())
    self.initialItemEmbeddings = Parameter(self.createRandomEmbeddings())
    self.concatenatedUserEmbeddings = Parameter(self.createRandomEmbeddings())
    self.concatenatedItemEmbeddings = Parameter(self.createRandomEmbeddings())
    for name, parameter in self.named_parameters():
      if ('bias' not in name):
        init.xavier_uniform_(parameter)
    
    self.W1 = self.createWeightMatrix()
    self.W2 = self.createWeightMatrix()

  def createWeightMatrix(self):
    output = []
      output.append(Linear(transfomationSize, transfomationSize))
    return ModuleList(output)
  
  def createRandomEmbeddings():
    return torch.rand(numberOfUsers, transfomationSize)

  def createEmptyEmbeddings():
    return torch.zeros((numberOfUsers, transfomationSize * (numberOfLayers + 1)))

  def forward(self, users, positives, negatives):
    # Combines the user and item embeddings into one array
    initialEmbeddings = torch.cat((self.initialUserEmbeddings, self.initialItemEmbeddings))
    outputEmbeddings = [initialEmbeddings]

    # Does the messaging
    for l in range(numberOfLayers):
      lE = torch.sparse.mm(sparceAdjacenyMatrix, initialEmbeddings)
      initialEmbeddings = LeakyReLU()(self.W1[l](lE) + self.W2[l](initialEmbeddings.mul(lE)))
      normalizedEmbeddings = F.normalize(initialEmbeddings, p=2, dim=1)
      # Adds them to the list
      outputEmbeddings.append(normalizedEmbeddings)

    outputEmbeddings = torch.cat(outputEmbeddings, 1)
    
    userEmbeds, itemEmbeds = outputEmbeddings.split((numberOfUsers, numberOfItems), 0)
    self.concatenatedUserEmbeddings = Parameter(userEmbeds)
    self.concatenatedItemEmbeddings = Parameter(itemEmbeds)

    return self.getLoss(userEmbeds[users-1], itemEmbeds[positives], itemEmbeds[negatives])

  def getLoss(self, u, i, j):
    return -(torch.log(torch.sigmoid(torch.mul(u, i).sum(dim=1) - torch.mul(u, j).sum(dim=1)))).mean()

# Running

In [213]:
model = NGCF()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 10

model.train()
batchs = numberOfItems // batch_size + 1

def train(model, t):
  print('Epoch ' + str(t+1))
  total_loss = 0
  for _i in range(batchs):
    user, pos, neg = get_a_sample()
    optimizer.zero_grad()
    loss = model(torch.from_numpy(user).long(), torch.LongTensor(pos), torch.LongTensor(neg))
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(' Loss = ' + str(total_loss))

for t in range(epochs):
  train(model, t)
  model.train()
  model.eval()

Epoch 1
 Loss = 3.6117654144763947
Epoch 2
 Loss = 2.2050976306200027
Epoch 3
 Loss = 1.8743716776371002
Epoch 4
 Loss = 1.6849270910024643
Epoch 5
 Loss = 1.5634060502052307
Epoch 6
 Loss = 1.5372567921876907
Epoch 7
 Loss = 1.406380757689476
Epoch 8
 Loss = 1.3347729742527008
Epoch 9
 Loss = 1.2633979618549347
Epoch 10
 Loss = 1.1762020513415337


# Evaluating the model

In [214]:
# Gets embedings from model
user_embeddings = model.concatenatedUserEmbeddings.detach()
item_embeddings = model.concatenatedItemEmbeddings.detach()

# Calculates precision and recall
recall_parts, ndcg_parts = [], []

non_train_items = (1 - user_embeddings).float()
predictions = torch.mm(user_embeddings, item_embeddings.t())
predictions = predictions * (1-userItemIteractionMatrix)

output = np.zeros((numberOfUsers, numberOfItems))
reversedMatrix = 1-userItemIteractionMatrix
top10 = torch.topk(predictions * reversedMatrix, k=10).indices
for u in range(len(top10)):
  for i in top10[u]:
    output[u][i] = 1

print("Evaluating...")

truePositives = []
falsePositive = []
falseNegatives = []
for user in range(len(output)):
  TPtemp = 0
  FPtemp = 0
  FNtemp = 0
  for i in range(len(output[user])):
    if output[user][i] == 1 and testingUIMatrix[user][i] == 1:
      TPtemp += 1
    elif testingUIMatrix[user][i] == 1 and output[user][i] == 0: #false negative
      FNtemp += 1
    elif testingUIMatrix[user][i] == 1 and output[user][i] == 0: #false positive
      FPtemp += 1
  truePositives.append(TPtemp)
  falsePositive.append(FPtemp)
  falseNegatives.append(FNtemp)

precision = 0
recall = 0
TPs = 0
for i in range(len(truePositives)):
  if (truePositives[i] + falsePositive[i]) != 0:
    precision += truePositives[i]/(truePositives[i] + falsePositive[i])
  if (truePositives[i] + falseNegatives[i]) != 0:
    recall += truePositives[i]/(truePositives[i] + falseNegatives[i])
  TPs += truePositives[i]
  
precision = precision/len(truePositives)
recall = recall/len(truePositives)
falseNegatives = TPs/len(falseNegatives)

print('Precision:\t' + str(precision))
print('Recall:\t\t' + str(recall))